In [2]:
!pip install torch

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/e4/7e/2f5e867229b4f96c21d1de759ecd2589a1468342ed570ef275bcd02f677f/torch-2.2.1-cp312-cp312-manylinux1_x86_64.whl.metadata
  Obtaining dependency information for filelock from https://files.pythonhosted.org/packages/8b/69/acdf492db27dea7be5c63053230130e0574fd8a376de3555d5f8bbc3d3ad/filelock-3.13.3-py3-none-any.whl.metadata
  Obtaining dependency information for sympy from https://files.pythonhosted.org/packages/d2/05/e6600db80270777c4a64238a98d442f0fd07cc8915be2a1c16da7f2b9e74/sympy-1.12-py3-none-any.whl.metadata
  Obtaining dependency information for networkx from https://files.pythonhosted.org/packages/d5/f0/8fbc882ca80cf077f1b246c0e3c3465f7f415439bdea6b899f6b19f61f70/networkx-3.2.1-py3-none-any.whl.metadata
  Obtaining dependency information for fsspec from https://files.pythonhosted.org/packages/93/6d/66d48b03460768f5

In [4]:
!pip install torchvision

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for torchvision from https://files.pythonhosted.org/packages/62/70/bb489d4d8ff75eb1b59094b652d3137374e9c05d1646146e8f927c513051/torchvision-0.17.1-cp312-cp312-manylinux1_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 12.9 MB/s eta 0:00:00m eta 0:00:010:01:01


In [5]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST

# Hyperparameters
input_size = 28 * 28  # MNIST images are 28x28 pixels
hidden_sizes = [128, 64]  # Two hidden layers with 128 and 64 neurons respectively
output_size = 10  # 10 classes for MNIST digits
batch_size = 64
learning_rate = 0.001
num_epochs = 10

# Define a Feedforward Neural Network
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_sizes[0])
        self.fc2 = nn.Linear(hidden_sizes[0], hidden_sizes[1])
        self.fc3 = nn.Linear(hidden_sizes[1], output_size)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)  # Softmax for multiclass classification

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        x = self.softmax(x)
        return x

# Preprocessing of input images
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert PIL Image or numpy.ndarray to tensor
    transforms.Normalize((0.5,), (0.5,))  # Normalize images with mean and std deviation
])

# Load MNIST dataset
train_dataset = MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = MNIST(root='./data', train=False, transform=transform)

# Data loaders
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# Initialize the neural network
model = NeuralNetwork(input_size, hidden_sizes, output_size)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Cross Entropy Loss for classification
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Reshape images to (batch_size, input_size)
        images = images.reshape(-1, input_size)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_step}], Loss: {loss.item():.4f}')

# Testing the model
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, input_size)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy of the network on the 10000 test images: {100 * correct / total}%')


100%|████████████████████████████| 9912422/9912422 [00:01<00:00, 7012344.09it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|███████████████████████████████| 28881/28881 [00:00<00:00, 50494245.03it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|████████████████████████████| 1648877/1648877 [00:00<00:00, 2425711.90it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████████████████████████████| 4542/4542 [00:00<00:00, 9426288.36it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Epoch [1/10], Step [100/938], Loss: 1.6919
Epoch [1/10], Step [200/938], Loss: 1.6522
Epoch [1/10], Step [300/938], Loss: 1.5569
Epoch [1/10], Step [400/938], Loss: 1.5293
Epoch [1/10], Step [500/938], Loss: 1.5710
Epoch [1/10], Step [600/938], Loss: 1.5401
Epoch [1/10], Step [700/938], Loss: 1.6163
Epoch [1/10], Step [800/938], Loss: 1.5194
Epoch [1/10], Step [900/938], Loss: 1.5469
Epoch [2/10], Step [100/938], Loss: 1.5503
Epoch [2/10], Step [200/938], Loss: 1.5056
Epoch [2/10], Step [300/938], Loss: 1.5723
Epoch [2/10], Step [400/938], Loss: 1.4974
Epoch [2/10], Step [500/938], Loss: 1.4822
Epoch [2/10], Step [600/938], Loss: 1.5735
Epoch [2/10], Step [700/938], Loss: 1.5243
Epoch [2/10], Step [800/938], Loss: 1.5264
Epoch [2/10], Step [900/938], Loss: 1.5327
Epoch [3/10], Step [100/938], Loss: 1.5171
Epoch [3/10], Step [200/938], Loss: 1.5456
Epoch [3/10], Step [300/938], Loss: 1.5394
Epoch [3/10], Step [40